In [1]:
from pathlib import Path
import re
import xarray as xr
import virtualizarr
from typing import List
import logging

In [2]:
logging.basicConfig()
logger = logging.getLogger("catalog_netcdf")
logger.setLevel(logging.DEBUG)

In [3]:
input_dir = Path(
    "/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/"
)
storage_prefix = Path("/scratch/k/k202134/virtual_datasets")
storage_prefix.mkdir(exist_ok=True)

In [4]:
files = input_dir.glob("*.nc")
stems = list(f.stem for f in files)
s0 = stems[0]
for shared in range(1, len(s0) + 1):
    if not all(x.startswith(s0[:shared]) for x in stems):
        shared -= 1
        break
id = s0[:shared]
parts = [x[shared:] for x in stems]

In [5]:
patterns = {
    re.sub(r"\d{4}-\d{2}-\d{2}", "", x) for x in parts
}  # r'\\d\{4\}-\\d\{2\}-\\d\{2\}'
patterns = {re.sub(r"\d{8}T\d{6}Z", "", x) for x in patterns}  # r'\\d\{8\}T\\d\{6\}Z'
patterns = {re.sub(r"^_", "", x) for x in patterns}
patterns = {re.sub(r"_$", "", x) for x in patterns}
patterns

{'atm_2d_ml',
 'atm_3d_ml',
 'atm_mon',
 'hd_meanflow',
 'oce_fx',
 'oce_mon',
 'oce_qps'}

In [6]:
filelist = {
    pattern: sorted(input_dir.glob(f"{id}*{pattern}*.nc")) for pattern in patterns
}

In [7]:
def get_subset_info(datasets):  # currently not used, but code might be useful.
    dimsets = {pattern: ds.sizes for pattern, ds in datasets.items()}
    subsets = {
        dataset: {x: 0 for x, count in dims.items() if count == 1}
        for dataset, dims in dimsets.items()
    }
    subsetted = {
        name: datasets[name].isel(subset).drop_vars(subset.keys())
        for name, subset in subsets.items()
    }
    subset_dims = {name: ds.sizes for name, ds in subsetted.items()}
    subset_dims

In [9]:
def load_virtual_dataset(name: str, files: List[Path]):
    to_load = variables_to_load(files)
    logger.debug(f"loading {to_load}")
    logger.debug(f"{files=}")
    virtual_datasets = [
        virtualizarr.open_virtual_dataset(
            filepath=str(filepath), loadable_variables=to_load, indexes={}
        )
        for filepath in files
    ]
    logger.debug("combining virtual datasets")
    virtual_ds = xr.combine_nested(
        virtual_datasets, concat_dim=["time"], coords="minimal", compat="override"
    )
    return virtual_ds


def variables_to_load(files) -> List[str]:
    ds = xr.open_mfdataset(files)
    to_load = [
        x
        for x in ds.variables
        if (len([dim for dim in ds[x].shape if dim > 1]) <= 1) | ds[x].size < 1e6
    ]
    return to_load

In [10]:
%%time

for name, files in filelist.items():
    logger.info(f"loading {name}")
    virtual_ds = load_virtual_dataset(name, files)
    print(type(virtual_ds))
    logger.info(f"writing {name}")
    virtual_ds.virtualize.to_kerchunk(
        storage_prefix / Path(f"{name}.parq"), format="parquet"
    )

INFO:catalog_netcdf:loading oce_mon
DEBUG:catalog_netcdf:loading ['total_salt', 'total_saltinseaice', 'total_saltinliquidwater', 'amoc26n', 'kin_energy_global', 'pot_energy_global', 'total_energy_global', 'ssh_global', 'sst_global', 'sss_global', 'potential_enstrophy_global', 'HeatFlux_Total_global', 'FrshFlux_Precipitation_global', 'FrshFlux_SnowFall_global', 'FrshFlux_Evaporation_global', 'FrshFlux_Runoff_global', 'FrshFlux_VolumeIce_global', 'FrshFlux_TotalOcean_global', 'FrshFlux_TotalIce_global', 'FrshFlux_VolumeTotal_global', 'totalsnowfall_global', 'ice_volume_nh', 'ice_volume_sh', 'ice_extent_nh', 'ice_extent_sh', 'global_heat_content', 'global_heat_content_solid', 'time', 'lon', 'lat']
DEBUG:catalog_netcdf:files=[PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_oce_mon_17000101T000000Z.nc'), PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel2431

<class 'xarray.core.dataset.Dataset'>


INFO:catalog_netcdf:loading atm_2d_ml
DEBUG:catalog_netcdf:loading ['time', 'clon', 'clat', 'height', 'height_2']
DEBUG:catalog_netcdf:files=[PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_atm_2d_ml_17000101T000000Z.nc'), PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_atm_2d_ml_17050101T000000Z.nc'), PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_atm_2d_ml_17100101T000000Z.nc'), PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_atm_2d_ml_17150101T000000Z.nc'), PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_atm_2d_ml_17200101T000000Z.nc'), Pos

<class 'xarray.core.dataset.Dataset'>


INFO:catalog_netcdf:loading oce_qps
DEBUG:catalog_netcdf:loading ['global_hfl', 'atlantic_hfl', 'pacific_hfl', 'global_wfl', 'atlantic_wfl', 'pacific_wfl', 'global_hfbasin', 'atlantic_hfbasin', 'pacific_hfbasin', 'global_sltbasin', 'atlantic_sltbasin', 'pacific_sltbasin', 'time', 'clon', 'clat', 'lon', 'lat', 'depth', 'depth_2', 'lev']
DEBUG:catalog_netcdf:files=[PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_oce_qps_17000101T000000Z.nc'), PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_oce_qps_17050101T000000Z.nc'), PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_oce_qps_17100101T000000Z.nc'), PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312

...

<class 'xarray.core.dataset.Dataset'>


INFO:catalog_netcdf:loading atm_3d_ml
DEBUG:catalog_netcdf:loading ['height_bnds', 'depth_bnds', 'depth_2_bnds', 'time', 'clon', 'clat', 'height', 'depth', 'depth_2']
DEBUG:catalog_netcdf:files=[PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_atm_3d_ml_17000101T000000Z.nc'), PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_atm_3d_ml_17050101T000000Z.nc'), PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_atm_3d_ml_17100101T000000Z.nc'), PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_atm_3d_ml_17150101T000000Z.nc'), PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/he

<class 'xarray.core.dataset.Dataset'>


INFO:catalog_netcdf:loading hd_meanflow
DEBUG:catalog_netcdf:loading ['time', 'lon', 'lat']
DEBUG:catalog_netcdf:files=[PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_1700-01-02_hd_meanflow.nc'), PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_1705-01-02_hd_meanflow.nc'), PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_1710-01-02_hd_meanflow.nc'), PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_1715-01-02_hd_meanflow.nc'), PosixPath('/work/mh0033/m211054/projects/icon/seamless/icon-2024.10/build_hdext/experiments/hel24312_r5b7_ctrl/outdata/hel24312_r5b7_ctrl_1720-01-02_hd_meanflow.nc'), PosixPath('/work/mh0033/m211054/projects/icon

<class 'xarray.core.dataset.Dataset'>
CPU times: user 8min 17s, sys: 7min 34s, total: 15min 51s
Wall time: 1h 8min 32s


In [ ]:
%%time
xr.open_dataset("/scratch/k/k202134/oce_fx.parq", engine="kerchunk", chunks={})